# Econometric Feature Creation

In [1]:
#imports 
import pandas as pd
import numpy as np
import os
import statsmodels.formula.api as sm
import pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange, tqdm
import statsmodels.api as sm
import networkx as nx
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold

In [2]:
import pandas as pd
import requests

def get_sitc_codes():
    # URL of the JSON file
    url = 'https://comtradeapi.un.org/files/v1/app/reference/S4.json'

    try:
        # Send a GET request to the URL and fetch the data
        response = requests.get(url)
        response.raise_for_status()  # Check that the request was successful
        
        # Load the JSON data
        data = response.json()

        # Since the JSON data might be nested, use json_normalize with appropriate arguments
        if isinstance(data, list):
            # If the top level is a list
            df = pd.json_normalize(data)
        else:
            # If the top level is a dictionary
            # Identify the key that holds the main data (adjust the path as necessary)
            main_data_key = 'results'  # Adjust this based on the actual structure
            df = pd.json_normalize(data[main_data_key])

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
    except KeyError as e:
        print(f"Error processing JSON structure: {e}")

    return df

In [4]:
class CreateFeatures:
    """
    We define a class which builds the feature dataframe 
    """
    
    def __init__(self, year = 1962, data_dir = "../data/"):
        self.year = year
        self.data_dir = data_dir
        
    def prepare_econ_features(self, filter_gdp = True):
        
        #DATA IMPORT
        #import dictionary with all features from WB
        with open(self.data_dir + 'all_wb_indicators.pickle', 'rb') as handle:
            features_dict = pkl.load(handle)
            
        self.feature_list = list(features_dict.keys())[1:]
        #import list of all features we want to select for

        #look up each of the features -- add country feature in that year 
        i = 0
        for feature in self.feature_list:
            #find dataframe corresponding to specific feature name
            df = features_dict[feature]
            
            if (i == 0):
                self.features = df[["economy", "YR" + str(self.year)]]
            else: 
                self.features = pd.merge(self.features, 
                                            df[["economy", "YR" + str(self.year)]],
                                            on = "economy", how = "outer")
            self.features.rename(columns = {"YR" + str(self.year): feature}, inplace = True)
            i = i+1
        
        #prepare GDP feature
        self.gdp_growth = features_dict['NY.GDP.MKTP.KD.ZG']
        cols = list(self.gdp_growth.columns.copy())
        cols.remove("economy")
        self.gdp_growth["country_sd"] = self.gdp_growth[cols].std(axis=1)
        #select potential variables 
        self.gdp_growth["prev_gdp_growth"] = self.gdp_growth["YR" + str(self.year-1)]
        self.gdp_growth["current_gdp_growth"] = self.gdp_growth["YR" + str(self.year)]
        #we eliminate countries that are too volatile in growth -- probably an indicator that growth estimates are inaccurate
        self.gdp_growth = self.gdp_growth[["economy", "prev_gdp_growth",
                                "current_gdp_growth"]].dropna()
        
        #combine GDP and other features
        self.features = pd.merge(self.gdp_growth, self.features,
                                   on = "economy", how = "left")
        #we only keep countries where we observe GDP growth -- otherwise nothing to predict
        #we keep countries where other features may be missing -- and fill NAs with 0 
        self.features.rename(columns = {"economy": "country_code"}, inplace = True)
        
    def prepare_network_features(self):
        """
        We create an initial, import-centric trade link pandas dataframe for a given year
        """
        #get product codes
        data_dict = get_sitc_codes()
        data_cross = []
        i = 0
        for item_def in list(data_dict["text"]):
            if(i >= 2):
                data_cross.append(item_def.split(" - ", 1))
            i = i+1

        self.product_codes = pd.DataFrame(data_cross, columns = ['code', 'product'])
        self.product_codes["sitc_product_code"] = self.product_codes["code"]
        
        #get country codes
        self.country_codes = pd.read_excel(self.data_dir + "ISO3166.xlsx")
        self.country_codes["location_code"] = self.country_codes["Alpha-3 code"]
        self.country_codes["partner_code"] = self.country_codes["Alpha-3 code"]
        self.country_codes["country_i"] = self.country_codes["English short name"]
        self.country_codes["country_j"] = self.country_codes["English short name"]
        
        #get trade data for a given year
        trade_data = pd.read_stata(self.data_dir + "country_partner_sitcproduct4digit_year_"+ str(self.year)+".dta") 
        #merge with product / country descriptions
        trade_data = pd.merge(trade_data, self.country_codes[["location_code", "country_i"]],on = ["location_code"])
        trade_data = pd.merge(trade_data, self.country_codes[["partner_code", "country_j"]],on = ["partner_code"])
        trade_data = pd.merge(trade_data, self.product_codes[["sitc_product_code", "product"]], 
                              on = ["sitc_product_code"])
        ###select level of product aggregation
        trade_data["product_category"] = trade_data["sitc_product_code"].apply(lambda x: x[0:1])
        #trade_data = trade_data[trade_data["product_category"] == "1"]
        
        #keep only nodes that we have features for
        trade_data = trade_data[trade_data["location_code"].isin(self.features["country_code"])]
        trade_data = trade_data[trade_data["partner_code"].isin(self.features["country_code"])]
        
        if (len(trade_data.groupby(["location_code", "partner_code", "sitc_product_code"])["import_value"].sum().reset_index()) != len(trade_data)):
            print("import, export, product combination not unique!")
        self.trade_data1 = trade_data
        #from import-export table, create only import table
        #extract imports
        imports1 = trade_data[['location_id', 'partner_id', 'product_id', 'year',
               'import_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        imports1 = imports1[imports1["import_value"] != 0]
        #transform records of exports into imports
        imports2 = trade_data[['location_id', 'partner_id', 'product_id', 'year',
               'export_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        imports2["temp1"] = imports2['partner_code']
        imports2["temp2"] = imports2['location_code']

        imports2['location_code'] = imports2["temp1"]
        imports2['partner_code'] = imports2["temp2"]
        imports2["import_value"] = imports2["export_value"]
        imports2 = imports2[imports2["import_value"] != 0]
        imports2 = imports2[['location_id', 'partner_id', 'product_id', 'year',
               'import_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        
        imports_table = pd.concat([imports1, imports2]).drop_duplicates()
        
        #rename columns for better clarity
        imports_table["importer_code"] = imports_table["location_code"]
        imports_table["exporter_code"] = imports_table["partner_code"]
        imports_table["importer_name"] = imports_table["country_i"]
        imports_table["exporter_name"] = imports_table["country_j"]
        
        cols = ["importer_code", "exporter_code", "importer_name", "exporter_name",
               'product_id', 'year', 'import_value', 'sitc_eci', 'sitc_coi',
               'sitc_product_code', 'product', "product_category"]
        imports_table = imports_table[cols]
        
        exporter_total = imports_table.groupby(["exporter_code"])["import_value"].sum().reset_index()
        exporter_total = exporter_total.rename(columns = {"import_value": "export_total"})
        
        importer_total = imports_table.groupby(["importer_code"])["import_value"].sum().reset_index()
        importer_total = importer_total.rename(columns = {"import_value": "import_total"})
        
        ##### COMPUTE CENTRALITY FOR COUNTRY
        #sum imports across all products between countries into single value 
        imports_table_grouped = imports_table.groupby(["importer_code", "exporter_code"])["import_value"].sum().reset_index()
        imports_table_grouped = pd.merge(imports_table_grouped, importer_total, on = "importer_code")
        imports_table_grouped["import_fraction"] = imports_table_grouped["import_value"]\
                        /imports_table_grouped["import_total"]*100
        
        self.trade_data = imports_table_grouped
        
        #filter features and nodes to ones that are connected to others in trade data
        list_active_countries = list(set(list(self.trade_data ["importer_code"])+\
                        list(self.trade_data ["exporter_code"])))
        self.features = self.features[self.features["country_code"].isin(list_active_countries)].reset_index()
        self.features["node_numbers"] = self.features.index
        
        G=nx.from_pandas_edgelist(self.trade_data, 
                          "exporter_code", "importer_code", create_using = nx.DiGraph())
        
        self.G = G
        self.centrality_overall= nx.eigenvector_centrality(G, max_iter= 10000) 
        self.centrality_overall = pd.DataFrame(list(map(list, self.centrality_overall.items())), 
                                               columns = ["country_code", "centrality_overall"])
        G=nx.from_pandas_edgelist(self.trade_data, 
                          "exporter_code", "importer_code", ["import_fraction"])
        weighted_centrality = nx.eigenvector_centrality(G, weight = "import_fraction", max_iter= 10000) 
        weighted_centrality  = pd.DataFrame(list(map(list, weighted_centrality.items())), 
                                               columns = ["country_code", "weighted_centrality"])
        self.centrality_overall = pd.merge(self.centrality_overall, weighted_centrality, on = "country_code")
        
                               
        ##### COMPUTE CENTRALITY FOR COUNTRY IN PRODUCT CATEGORIES

        #sum imports across all products between countries into single value 
        imports_table_grouped = imports_table.groupby(["importer_code", "exporter_code"])["import_value"].sum().reset_index()
        products_grouped = imports_table.groupby(["product_category"])["import_value"].sum().reset_index()
        products_grouped = products_grouped.rename(columns = {"import_value": "import_product_total"})
        
        #sum exports in each category 
        self.export_types = imports_table.groupby(["importer_code", "exporter_code", "product_category"])["import_value"].sum().reset_index()
        self.export_types = pd.merge(products_grouped, self.export_types, on = "product_category")
        
        self.export_types["product_export_fraction"] = self.export_types["import_value"]\
                                                    /self.export_types["import_product_total"]*100
        
        list_products = list(set(self.export_types["product_category"]))
        
        i = 0 
        for product in list_products:
            
            temp = self.export_types[self.export_types["product_category"] == product].copy()
            
            G_w=nx.from_pandas_edgelist(temp, 
                "exporter_code", "importer_code", ["product_export_fraction"], create_using = nx.DiGraph())
            centrality_product_w = nx.eigenvector_centrality(G_w, weight = "product_export_fraction", 
                                                           max_iter= 10000)

            G=nx.from_pandas_edgelist(temp,"exporter_code", "importer_code", create_using = nx.DiGraph())
            centrality_product = nx.eigenvector_centrality(G,max_iter= 10000)

            if(i == 0):
                self.centrality_product = pd.DataFrame(list(map(list, centrality_product.items())), 
                                               columns = ["country_code", "prod_" + product])
                

            else: 
                self.centrality_product = pd.merge(self.centrality_product, 
                                               pd.DataFrame(list(map(list, centrality_product.items())), 
                                               columns = ["country_code", "prod_" + product]), 
                                                  on = "country_code")
                
            self.centrality_product = pd.merge(self.centrality_product, 
                                               pd.DataFrame(list(map(list, centrality_product_w.items())), 
                                               columns = ["country_code", "prod_w_" + product]), 
                                                  on = "country_code")
            
            i = i+1         
    
    def combine_normalize_features(self):
        
        self.combined_features = pd.merge(self.features, self.centrality_overall,on = "country_code")
        self.combined_features = pd.merge(self.combined_features, self.centrality_product,on = "country_code")
        #step eliminates NA and nodes that are not in graph, since they will have NA for graph features
        self.combined_features = self.combined_features.drop(columns = ["index"])
        #filter both trade data and features data to same subset of countries
        self.combined_features = self.combined_features[\
                                self.combined_features.country_code.isin(self.trade_data.importer_code)|\
                                self.combined_features.country_code.isin(self.trade_data.exporter_code)]
        self.trade_data = self.trade_data[\
                          self.trade_data.importer_code.isin(self.combined_features.country_code)&\
                          self.trade_data.exporter_code.isin(self.combined_features.country_code)]
        
        features_to_norm = list(self.combined_features.columns.copy())
        non_norm = ["country_code", "node_numbers"]
        cols_insufficient_data = list(self.combined_features.loc[:, self.combined_features.nunique() < 2].columns.copy())
        non_norm.extend(cols_insufficient_data)
 
        features_to_norm = [x for x in features_to_norm if x not in non_norm]
        scaler = StandardScaler()
        #we preserve NAs in the scaling
        self.combined_features[features_to_norm] = scaler.fit_transform(self.combined_features[features_to_norm])
        self.combined_features.fillna(0, inplace = True) #we fill NA after scaling 
        #check that feature has at least 20% coverage in a given year -- otherwise set to NA
        for feature in self.feature_list:
            coverage = len(self.combined_features[self.combined_features[feature] != 0])/len(self.combined_features)
            if(coverage <= 0.20): self.combined_features[feature] = 0

    def combine_features(self):
        self.combined_features = pd.merge(self.features, self.centrality_overall,on = "country_code")
        self.combined_features = pd.merge(self.combined_features, self.centrality_product,on = "country_code")
        #step eliminates NA and nodes that are not in graph, since they will have NA for graph features
        self.combined_features = self.combined_features.drop(columns = ["index"])
        #filter both trade data and features data to same subset of countries
        self.combined_features = self.combined_features[\
                                self.combined_features.country_code.isin(self.trade_data.importer_code)|\
                                self.combined_features.country_code.isin(self.trade_data.exporter_code)]
        self.trade_data = self.trade_data[\
                          self.trade_data.importer_code.isin(self.combined_features.country_code)&\
                          self.trade_data.exporter_code.isin(self.combined_features.country_code)]
        
        features_to_norm = list(self.combined_features.columns.copy())
        non_norm = ["country_code", "node_numbers"]
        cols_insufficient_data = list(self.combined_features.loc[:, self.combined_features.nunique() < 2].columns.copy())
        
        self.combined_features.fillna(0, inplace = True) #we fill NA after scaling 
        #check that feature has at least 20% coverage in a given year -- otherwise set to NA
        for feature in self.feature_list:
            coverage = len(self.combined_features[self.combined_features[feature] != 0])/len(self.combined_features)
            if(coverage <= 0.20): self.combined_features[feature] = 0

In [16]:
data_dict = {}

#GDP data goes up to 2018 -- trange up to 2019 spans up to 2018
for year in trange(1962, 2019):
    trade = CreateFeatures(year = year)
    trade.prepare_econ_features()
    trade.prepare_network_features()
    #trade.combine_normalize_features()
    trade.combine_features()
    
    data_dict[year] = trade

  0%|          | 0/57 [00:00<?, ?it/s]/var/folders/qh/0wyfgfcs3399gr_m2wrrtnzc0000gn/T/ipykernel_71000/3418551092.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.features.rename(columns = {"YR" + str(self.year): feature}, inplace = True)
  2%|▏         | 1/57 [00:18<17:37, 18.89s/it]/var/folders/qh/0wyfgfcs3399gr_m2wrrtnzc0000gn/T/ipykernel_71000/3418551092.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.features.rename(columns = {"YR" + str(self.year): feature}, inplace = True)
  4%|▎         | 2/57 [00:36<16:44, 18.27s/it]/var/folders/qh/0wyfgfcs3399gr_m2wrrtnzc0000gn/T/ipykernel_71000/3418551092.

In [17]:
data_dict[2000].combined_features

,country_code,prev_gdp_growth,current_gdp_growth,AG.CON.FERT.PT.ZS,AG.CON.FERT.ZS,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,...,prod_3,prod_w_3,prod_0,prod_w_0,prod_1,prod_w_1,prod_2,prod_w_2,prod_7,prod_w_7
0,ABW,1.238042,7.616589,0.000000,0.000000,20.0,11.111111,2000.0,0.022446,11.111111,...,0.023039,2.721247e-04,0.039556,0.000930,0.045334,0.005069,0.018923,0.000485,0.034490,0.000384
1,AGO,2.181490,3.054624,0.000000,0.466667,469613.9,37.668557,3000000.0,0.182993,2.406353,...,0.024860,1.850995e-06,0.062602,0.003061,0.047013,0.001603,0.048055,0.000776,0.060270,0.001111
2,ALB,12.890804,6.946217,0.000000,32.352941,11440.0,41.751825,578000.0,0.187114,21.094891,...,0.034950,2.663992e-05,0.062196,0.001565,0.042195,0.001094,0.046592,0.000248,0.056867,0.000182
3,AND,4.099079,3.528362,0.000000,0.000000,230.0,48.936170,1000.0,0.015129,2.127660,...,0.021614,9.533180e-05,0.035824,0.001261,0.056871,0.005805,0.023662,0.000150,0.043347,0.000180
4,ARE,2.902214,10.852704,14.420707,585.000000,5470.0,7.702056,60000.0,0.018319,0.844832,...,0.083479,4.370664e-03,0.088472,0.026230,0.104377,0.023830,0.094025,0.005526,0.082058,0.017893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,WSM,2.185434,5.093045,0.000000,0.000000,480.0,16.961131,14000.0,0.076084,4.946996,...,0.009425,5.556124e-07,0.019600,0.000072,0.010781,0.000018,0.017377,0.000024,0.030078,0.000360
178,YEM,3.775530,6.181916,0.000000,9.836893,236690.0,44.830199,1545000.0,0.082937,2.926303,...,0.035316,2.087310e-06,0.065133,0.008367,0.043535,0.002652,0.052627,0.000357,0.060249,0.000787
179,ZAF,2.400000,4.200000,128.346047,53.598320,981250.0,80.888475,13812000.0,0.295045,11.385800,...,0.114060,1.589874e-02,0.097188,0.009200,0.105679,0.011048,0.109352,0.010731,0.113610,0.017410
180,ZMB,4.650190,3.897323,3487.888889,11.147372,224980.0,30.264061,2816000.0,0.284699,3.788052,...,0.026082,1.510854e-05,0.042422,0.000081,0.024728,0.000096,0.058245,0.000199,0.074866,0.000211


In [18]:
with open("../feature_dicts/features_dict_nonnorm.pkl", "wb") as f:
    pkl.dump(data_dict, f)

In [6]:
with open("../feature_dicts/feature_dict_logged.pkl", "rb") as f:
    data_dict = pkl.load(f)

In [6]:
# Initialize variables to keep track of the DataFrame with the most rows
max_rows = 0
df_with_max_rows = None

# Iterate over the dictionary
for key, df in data_dict.items():
    # If this DataFrame has more rows than the current maximum, update the maximum and the DataFrame
    if len(df.combined_features) > max_rows:
        max_rows = len(df.combined_features)
        df_with_max_rows = df.combined_features
        max_year = key

In [7]:
# Compute the variance of each column
numeric_columns = df_with_max_rows.drop(['country_code'], axis=1)
variances = numeric_columns.var()

# Find columns with variance less than 0.1 (this is the threshold, adjust as needed)
columns_to_drop = variances[variances < 0.1].index
filtered_df = df_with_max_rows.drop(columns_to_drop, axis=1)

In [8]:
# Iterate over the dictionary
for key, df in data_dict.items():
    df.combined_features = df.combined_features.drop(columns_to_drop, axis=1)

## Mutual Information Selection

In [9]:
X_train = filtered_df.drop(['country_code','current_gdp_growth'], axis = 1)
Y_train = filtered_df['current_gdp_growth']

In [10]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectPercentile

mutual_info = mutual_info_regression(X_train, Y_train)

mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

selected_top_columns = SelectPercentile(mutual_info_regression, percentile=2)
selected_top_columns.fit(X_train, Y_train)
selected_top_columns.get_support()

array([False, False, False, ..., False, False, False])

In [11]:
columns = X_train.columns[selected_top_columns.get_support()]
X_train = X_train[columns]

In [12]:
column_list = list(columns)
column_list.append('current_gdp_growth')
column_list.append('country_code')

In [13]:
for key, df in data_dict.items():
    df.combined_features = df.combined_features[column_list]

In [14]:
columns_all_zeros = []
for key, df in data_dict.items():
    zero_columns = [col for col in df.combined_features.columns if (df.combined_features[col] == 0).all()]

    # Iterate over list1
    for item in zero_columns:
        # If the item is not in list2, add it
        if item not in columns_all_zeros:
            columns_all_zeros.append(item)


In [15]:
for key, df in data_dict.items():
    df.combined_features = df.combined_features.drop(columns_all_zeros, axis=1)

In [16]:
data_dict[1962].combined_features

,IT.MLT.MAIN.P2,NE.CON.PRVT.KD.ZG,NE.CON.TOTL.KD.ZG,NV.IND.TOTL.KD.ZG,NV.SRV.TOTL.KD.ZG,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.KD.ZG,NY.GNP.MKTP.KD.ZG,NY.GNP.PCAP.KD.ZG,SP.ADO.TFRT,SP.POP.2024.FE.5Y,SP.POP.2024.MA.5Y,SP.POP.6569.FE.5Y,SP.POP.6569.MA.5Y,SP.POP.65UP.MA.ZS,current_gdp_growth,country_code
0,-0.111774,0.000000,-1.024184,0.000000,0.000000,-1.112815,-1.038860,-0.703257,-0.630105,-0.892056,0.112493,-0.161362,0.282761,0.626574,0.204211,-1.112815,ARG
1,1.301831,-0.340188,-0.246697,0.000000,0.000000,-0.714648,-0.780316,0.000000,-0.351662,-1.104107,-1.529665,-1.375682,1.179255,1.004898,1.104184,-0.714648,AUS
2,0.097787,0.000000,0.000000,0.000000,0.000000,-0.463491,-0.153232,0.000000,0.000000,-0.882694,-0.717122,0.877274,2.809942,2.482537,2.321666,-0.463491,AUT
3,0.417067,0.000000,0.000000,0.000000,0.000000,0.011966,0.395375,0.000000,0.000000,-1.402846,-2.383251,-2.241161,2.496846,2.458060,2.431750,0.011966,BEL
4,-0.685382,-0.913001,-0.700270,0.000000,0.000000,-1.590323,-1.541320,-1.038805,-0.983784,0.733933,0.551797,0.097609,0.329067,0.341150,0.307165,-1.590323,BEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,-0.611217,0.000000,0.000000,0.000000,0.000000,0.078634,-0.011029,0.000000,0.000000,0.635345,0.119556,0.378782,-0.147779,-0.315724,-0.368509,0.078634,TUR
72,-0.129166,0.000000,0.102288,0.000000,0.000000,-1.246832,-1.099825,0.000000,0.000000,-0.759923,-0.112155,-0.163143,0.871415,1.106939,1.135215,-1.246832,URY
73,2.761470,0.000000,0.000000,0.000000,0.000000,0.176675,0.333370,0.000000,0.000000,-0.553262,-1.746300,-2.167133,1.292109,1.442964,1.559219,0.176675,USA
74,-0.495877,-0.245292,-0.328785,0.000000,0.000000,0.627946,0.396841,0.655605,0.504555,0.276462,0.416738,0.129565,-0.947292,-0.888139,-0.864482,0.627946,VEN


In [ ]:
mis_features = ["IT.MLT.MAIN.P2","NE.CON.PRVT.KD.ZG","NE.CON.TOTL.KD.ZG","NV.IND.TOTL.KD.ZG","NV.SRV.TOTL.KD.ZG","NY.GDP.MKTP.KD.ZG","NY.GDP.PCAP.KD.ZG","NY.GNP.MKTP.KD.ZG","NY.GNP.PCAP.KD.ZG","SP.ADO.TFRT","SP.POP.2024.FE.5Y","SP.POP.2024.MA.5Y","SP.POP.6569.FE.5Y","SP.POP.6569.MA.5Y","SP.POP.65UP.MA.ZS"]

In [16]:
with open("../data/filtered_features_dict.pkl", "wb") as f:
    pkl.dump(data_dict, f)

In [8]:
with open("../feature_dicts/mis_norm.pkl", "rb") as f:
    original_mis = pkl.load(f)

In [9]:
mis_columns = original_mis[1962].combined_features.columns.to_list()

In [11]:
mis_logged = data_dict
for i in data_dict:
    year = i
    year_dict = data_dict[year].combined_features
    mis_logged[year].combined_features = year_dict[mis_columns]

In [12]:
with open("../feature_dicts/mis_features_logged.pkl", "wb") as f:
    pkl.dump(mis_logged, f)

## Random Subset Selection

In [9]:
X_train = filtered_df.drop(['country_code','current_gdp_growth'], axis = 1).iloc[:,1:-23]

In [10]:
random_columns = X_train.sample(n=15, axis=1).columns

In [6]:
random_columns = ['GC.XPN.TOTL.CN', 'DC.DAC.DNKL.CD', 'ST.INT.DPRT', 'SP.POP.6064.FE.5Y',
       'SL.IND.EMPL.MA.ZS', 'DT.NFL.PRVT.CD', 'SE.XPD.CSEC.ZS',
       'SP.URB.TOTL.IN.ZS', 'NY.GNP.MKTP.PP.KD', 'DC.DAC.POLL.CD',
       'TX.VAL.MRCH.R5.ZS', 'SE.PRM.OENR.FE.ZS', 'NE.CON.GOVT.CN',
       'SL.TLF.BASC.ZS', 'EN.ATM.PM25.MC.M3', 'country_code', 'current_gdp_growth']

In [11]:
# Iterate over the dictionary
for key, df in data_dict.items():

    # Combine the lists
    all_cols = ['country_code','current_gdp_growth'] + random_columns.tolist()

    # Select the columns from the dataframe
    df.combined_features = df.combined_features[all_cols]

In [16]:
with open("../feature_dicts/random_features_dict.pkl", "wb") as f:
    pkl.dump(data_dict, f)

In [6]:
with open("../feature_dicts/feature_dict_logged.pkl", "rb") as f:
    data_dict = pkl.load(f)

In [7]:
random_not_norm = data_dict
for i in enumerate(data_dict):
    year = i[1]
    year_dict = data_dict[year].combined_features
    random_not_norm[year].combined_features = year_dict[random_columns]

In [8]:
with open("../feature_dicts/random_dict_not_norm.pkl", "wb") as f:
    pkl.dump(random_not_norm, f)

## Logging of Data

In [15]:
data_dict[1962].combined_features

,country_code,prev_gdp_growth,current_gdp_growth,AG.CON.FERT.PT.ZS,AG.CON.FERT.ZS,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,...,prod_3,prod_w_3,prod_0,prod_w_0,prod_1,prod_w_1,prod_2,prod_w_2,prod_7,prod_w_7
0,ARG,5.427843,-0.852022,211.850000,0.662722,1364340.000,49.853655,19180000.0,0.912449,7.008466,...,0.180201,0.029555,0.111373,0.011093,0.131159,0.036479,0.138070,0.007523,0.141817,0.151116
1,AUS,2.482656,1.294611,98.105570,42.017703,4765929.315,62.037792,15515793.3,1.444405,2.019681,...,0.100081,0.013710,0.131789,0.022519,0.124412,0.179920,0.132209,0.055173,0.110967,0.153177
2,AUT,5.537979,2.648675,128.339112,172.626413,35310.000,42.789627,1681000.0,0.235769,20.370819,...,0.121553,0.013246,0.109839,0.019364,0.095939,0.034108,0.113209,0.021847,0.087408,0.020499
3,BEL,4.978423,5.212004,59.976855,0.000000,18160.000,55.332115,0.0,0.000000,30.591103,...,0.177465,0.103697,0.211117,0.155443,0.222559,0.144991,0.212290,0.153737,0.177582,0.119476
4,BEN,3.141280,-3.426410,0.000000,0.635106,14620.000,12.965591,940000.0,0.362472,8.336289,...,0.062381,0.000014,0.060122,0.000900,0.073466,0.002820,0.033260,0.000023,0.064829,0.000980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,TUR,1.156069,5.571429,121.564801,3.204747,366480.000,47.617686,23131000.0,0.796696,30.054702,...,0.132159,0.000885,0.078488,0.067527,0.082649,0.001222,0.118369,0.003908,0.113304,0.043985
72,URY,2.493038,-1.574549,655.000000,10.934891,162490.000,92.840818,2396000.0,0.924225,13.689864,...,0.075653,0.003196,0.096596,0.003355,0.111036,0.048396,0.104050,0.002546,0.097734,0.016413
73,USA,2.300000,6.100000,93.628184,46.725537,4439910.000,48.476137,177095000.0,0.949378,19.335711,...,0.235038,0.578704,0.214542,0.443631,0.224104,0.690762,0.216432,0.609340,0.211033,0.396553
74,VEN,3.192519,8.532934,800.000000,5.517241,193020.000,21.883113,2900000.0,0.331274,3.287795,...,0.110830,0.005863,0.140741,0.071596,0.145501,0.013035,0.145105,0.013939,0.132914,0.092288


In [18]:
data_dict_logged = data_dict
for year in data_dict:
    year_dict = data_dict[year].combined_features
    for column in year_dict.columns:
        if column != 'country_code':
            year_dict[column] = np.log(year_dict[column])
    data_dict_logged[year].combined_features = year_dict

/Users/sinclaireschuetze/Documents/GitHub/Trade-GNN-ChangePoint/.env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sinclaireschuetze/Documents/GitHub/Trade-GNN-ChangePoint/.env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [20]:
for year in data_dict_logged:
    year_dict = data_dict_logged[year].combined_features
    year_dict.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

In [23]:
with open("../feature_dicts/feature_dict_logged.pkl", "wb") as f:
    pkl.dump(data_dict_logged, f)

In [6]:
with open("../feature_dicts/feature_dict_logged.pkl", "rb") as f:
    data_dict_logged = pkl.load(f)

In [6]:
with open("../feature_dicts/mis_features_not_norm.pkl", "rb") as f:
    original_mis = pkl.load(f)

mis_columns = original_mis[1962].combined_features.columns.to_list()

In [7]:
mis_logged = data_dict
for year in data_dict:
    year_dict = data_dict[year].combined_features
    filtered_dict = year_dict[mis_columns]

    for column in filtered_dict.columns:
        if column != 'country_code':
            filtered_dict[column] = np.log(filtered_dict[column])
            
    filtered_dict.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
    mis_logged[year].combined_features = filtered_dict

/Users/sinclaireschuetze/Documents/GitHub/Trade-GNN-ChangePoint/.env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/qh/0wyfgfcs3399gr_m2wrrtnzc0000gn/T/ipykernel_7394/3971564832.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dict[column] = np.log(filtered_dict[column])
/Users/sinclaireschuetze/Documents/GitHub/Trade-GNN-ChangePoint/.env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/qh/0wyfgfcs3399gr_m2wrrtnzc0000gn/T/ipykernel_7394/3971564832.py:10: SettingWithCopyWarning: 
A value

In [8]:
mis_logged[1962].combined_features

,IT.MLT.MAIN.P2,NE.CON.PRVT.KD.ZG,NE.CON.TOTL.KD.ZG,NV.IND.TOTL.KD.ZG,NV.SRV.TOTL.KD.ZG,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.KD.ZG,NY.GNP.MKTP.KD.ZG,NY.GNP.PCAP.KD.ZG,SP.ADO.TFRT,SP.POP.2024.FE.5Y,SP.POP.2024.MA.5Y,SP.POP.6569.FE.5Y,SP.POP.6569.MA.5Y,SP.POP.65UP.MA.ZS,current_gdp_growth,country_code
0,1.496428,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,4.020303,2.075347,2.056642,0.904790,0.886136,1.605887,0.000000,ARG
1,2.724612,0.919234,1.004215,0.00000,0.00000,0.258210,0.000000,0.000000,0.000000,3.775057,1.877265,1.920734,1.255627,1.018098,1.980946,0.258210,AUS
2,1.802454,0.000000,0.000000,0.00000,0.00000,0.974060,0.703825,0.000000,0.000000,4.029859,1.980176,2.159822,1.686262,1.412064,2.333776,0.974060,AUT
3,2.140112,0.000000,0.000000,0.00000,0.00000,1.650964,1.567372,0.000000,0.000000,3.278201,1.756377,1.811191,1.616716,1.406656,2.360303,1.650964,BEL
4,-2.413655,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,5.001514,2.122299,2.083382,0.926265,0.773610,1.656637,0.000000,BEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,-0.422641,0.000000,0.000000,0.00000,0.00000,1.717651,1.008009,0.000000,0.000000,4.962873,2.076119,2.111627,0.679360,0.453270,1.263961,1.717651,TUR
72,1.466265,0.000000,1.625084,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,4.147395,2.050457,2.056455,1.148479,1.050898,1.991672,0.000000,URY
73,3.272853,0.000000,0.000000,0.00000,0.00000,1.808289,1.499772,0.000000,0.000000,4.318541,1.847940,1.821045,1.292199,1.151883,2.127753,1.808289,USA
74,0.428713,1.136650,0.778049,0.00000,0.00000,2.143933,1.568916,2.173471,1.619080,4.808070,2.108097,2.086632,0.045305,0.054214,0.828508,2.143933,VEN


In [9]:
with open("../feature_dicts/random_logged.pkl", "wb") as f:
    pkl.dump(mis_logged, f)